**<center><font size=5>(Bio)statistics in R: Part #3</font></center>**
***<center>"Check your Guesses with Hypothesis Testing"</center>***
***

**author**: Ruslan Klymentiev

**date**: 8th August, 2018

![](https://s22.postimg.cc/chgrosuqp/josh-couch-586382-unsplash.jpg)

$\blacktriangleright$ ***[Link to Part #1](https://www.kaggle.com/ruslankl/bio-statistics-in-r-part-1)*** 

$\blacktriangleright$ ***[Link to Part #2](https://www.kaggle.com/ruslankl/bio-statistics-in-r-part-2)***

- <a href='#intro'>Project Introduction</a>  
- <a href='#setup'>Setting up the Environment</a>
- <a href='#nhst'>Null Hypothesis Significance Testing</a> 
 - <a href='#nvsa'>Null vs Alternative Hypothesis</a> 
 - <a href='#ci'>Connection with CI</a> 
 - <a href='#pval'>P-value</a> 
- <a href='#power'>Statistical Power</a> 
 - <a href='#pwr'>Meet the pwr package</a> 
- <a href='#next'>What to Expect Next</a>

## <a id='setup'>Project Introduction</a> 

Alright! Now we know how to make assumptions about population when we have only the sample. We can run *t* and F tests and find confidence intervals. In this part we are going to ask questions about the sample and answer them with hypothesis testings. 

The dataset stays the same: [Healthcare Dataset](https://www.kaggle.com/asaumya/healthcare-dataset-stroke-data).

## <a id='setup'>Setting up the environment</a> 

In [1]:
suppressMessages(library(tidyverse))
suppressMessages(library(repr))
suppressMessages(library(pwr))
set.seed(123)
options(repr.plot.width=7, repr.plot.height=3)

Stroke_Data <- read.csv("healthcare-dataset-stroke-data/train_2v.csv")

# some data cleaning
Stroke_Data <- Stroke_Data %>% filter(gender != "Other" & !(is.na(bmi))) %>%
  select(-id)

# convert some variables to a factor
Stroke_Data$hypertension <- as.factor(Stroke_Data$hypertension)
Stroke_Data$heart_disease <- as.factor(Stroke_Data$heart_disease)
Stroke_Data$stroke <- as.factor(Stroke_Data$stroke)

sample_n(Stroke_Data, 5)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
12059,Female,38,0,0,Yes,Private,Urban,109.33,26.6,smokes,0
33054,Female,76,1,0,No,Private,Urban,208.70,32.9,never smoked,0
17148,Male,16,0,0,No,Private,Rural,96.73,33.8,,0
37024,Male,54,0,0,Yes,Self-employed,Urban,62.52,29.9,formerly smoked,0
39431,Male,46,0,0,Yes,Private,Rural,137.60,24.4,formerly smoked,0


## <a id='nhst'>Null Hypothesis Significance Testing</a> 

### <a id='nvsa'>Null vs Alternative Hypothesis</a> 

Hypothesis testing is one of the most fundamental techniques in statistics. You always want to ask question about your data - is the prevelance of disease among men higher rather than among women? Does Drug A help to reduce the pain better than Drug B? 

We will explore the most popular technique called **Null Hypothesis Significance Testing (NHST)**. According to this method we will always declare one hypothesis as status quo hypothesis, also called *null hypothesis*, $H_0$ - there is no evidence to assume that prevelance of disease differs among gender or pain relief ratio for Drug A is the same as for Drug B. The alternative hypothesis is usually labeled as $H_A$ or $H_1$ and that's the hypothesis we want to check. 

Calculations are performed assuming that the null hypothesis is true, and evidence is required to
reject the null hypothesis in favor of the alternative. There might be 4 different types of outcome:

* If $H_0$ is true and we failed to reject $H_0$, then we have **correctly accepted the null**;

* If $H_0$ is true and we rejected $H_0$, then we have made a **Type I error**;

* If $H_A$ is true and we rejected $H_0$, then we have **correctly rejected the null**;

* If $H_A$ is true and we failed to reject $H_0$, then we have made **Type II error**.

*P.S. To remember the difference between Type I error and Type II error I asways refer to this picture in my mind:*

![](https://chemicalstatistician.files.wordpress.com/2014/05/pregnant.jpg)

NHST controls the probabilities of the errors, *the error rates*, to make decisions. Specifically, the procedures ensure that the probability of a Type I error is small.

* The probability of correctly accepting the null is $1 − \alpha$;
* The probability of a Type I error is called $\alpha$.
* The probability of correctly rejecting the null is called Power or $1 − \beta$.
* The pobability of a Type II error is called $\beta$.

![](https://keydifferences.com/wp-content/uploads/2017/01/possible-outcomes.jpg)

Consider testing a mean parameter, $\mu$ via null hypothesis $H_0: \mu=\mu_0$. Consider an estimator of $\mu$, $S$, which will serve as your statistic. Assume that $S$ has standard error $SD(S)$, which is consistently estimated by $\hat{SD}$ and that $Z score$
\begin{gather}
Z=\frac{S-\mu_0}{\widehat{SD(S)}}
\end{gather}
either follows a normal distribution or limits to one via the central limit theorem. Then you reject $H_0$ under the
following rules:

* $H_A: \mu>\mu_0$: reject if $Z\geq Z_{1-\alpha}$ (*one-sided test*);

* $H_A: \mu<\mu_0$: reject if $Z\leq Z_{\alpha}$ (*one-sided test*);

* $H_A: \mu \neq \mu_0$: reject if $\vert Z \vert \geq Z_{1-\alpha/2}$ (*two-sided test*).

where $Z_\alpha$ is the $\alpha$ quantile of the standard normal; $\alpha$ - probability of rejecting the null hypothesis when it is correct (or as said before, type I error).

Here’s some examples of the test statistic and setting:

* Single mean: $Z=\frac{\sqrt{n}(\bar{X}-\mu_0)}{S}$

* Difference in two means: $Z=\frac{\bar{X_1}-\bar{X_2} - (\mu_1-\mu_2)}{\sqrt{S^2_1/n_1 + S^2_2/n_2}}$

* Binomial proportion: $Z=\frac{\sqrt{n}(\hat{p}-p_0)}{\hat{p}(1-\hat{p})}$

For data that can reasonably be assumed to be normal, the standardized statistic will follow a **Student’s t distribution** and one simply follows the same rules, but with the normal quantiles replaced by **t quantiles**.

***
**Here comes an example:**

We want two check wether the mean of average glucose level differs between men and women. 

* null hypothesis $H_0: \mu_1=\mu_2$

* alternative hypothesis $H_A: \mu_1 \neq \mu_2$ (two-sided test)

* $\alpha = 0.05$

In [2]:
## consider an example:
avg_glucose_level = Stroke_Data %>%
    group_by(gender) %>%
    summarise(mean = mean(avg_glucose_level), sd = sd(avg_glucose_level), n = n())
avg_glucose_level

gender,mean,sd,n
Female,102.5178,40.86181,24945
Male,105.2728,44.11804,16986


In [3]:
estimate = avg_glucose_level$mean[2] - avg_glucose_level$mean[1]
mu0 = 0
se = sqrt(avg_glucose_level$sd[2]^2 / avg_glucose_level$n[2] + avg_glucose_level$sd[1]^2 / avg_glucose_level$n[1])
ts = (estimate - mu0) / se
print(paste0("Calucated Z-score: ", round(ts, 2)))
print(paste0("Z_0.975: ", round(qnorm(.975), 2)))

[1] "Calucated Z-score: 6.47"
[1] "Z_0.975: 1.96"


**We can see that the resulted test statistic is much larger than 0.975 quantile of the normal distribution, so we reject null hypothesis.**
***
Now let's do one-sided test for single mean. We believe that average **BMI** for the population is larger than 30 (regardless of the gender). Let's check our beliefs:

* null hypothesis $H_0: \mu_1=\mu_0=30$

* alternative hypothesis $H_A: \mu_1 > \mu_0$ (one-sided test)

* $\alpha = 0.05$

We will reject the null hypothesis when $Z\geq Z_{1-\alpha}$

In [4]:
bmi = Stroke_Data %>%
    select(bmi) %>%
    summarise(mean = mean(bmi), sd = sd(bmi), n = n())
bmi

mean,sd,n
28.60516,7.770186,41931


In [5]:
mu_0 = 30
alpha=0.05
Z = sqrt(bmi$n)*(bmi$mean - mu_0)/bmi$sd

print(paste0("Calucated Z-score: ", round(Z, 2)))
print(paste0("Z_0.95: ", round(qnorm(1-alpha), 2)))

[1] "Calucated Z-score: -36.76"
[1] "Z_0.95: 1.64"


**Calculated score is much smaller than .95 quantile of the normal distribution, so we don't have enough evidence to reject the null hypothesis.**

### <a id='ci'>Connection with CI</a> 

Consider testing $H_0: \mu = \mu_0$ vs $H_A: \mu \neq \mu_0$. Take the set of all possible values for which you fail to reject $H_0$, this set is a $(1-\alpha)100$% CI for $\mu$. 

The same works in reverse, if a $(1-\alpha)100$% CI contains $\mu_0$ then we fail to reject $H_0$ (*the same as we did in t-tests*).

Looking back on previous example with average glucose level:

In [6]:
## 95% CI for difference in mean of average glucose level
## H_0: mu_1 - mu_2 = 0, H_A: mu_1 - mu_2 != 0
round(estimate + c(-1, 1) * qnorm(.975) * se, 2)

[1] 1.92 3.59

**95% CI doesn't contain a 0, so we reject the null hypothesis.**

Also, we could call `t.test` function which we have seen in part #1. We can see that results are the same:

In [7]:
## example 1 with average glucose level
t.test(Stroke_Data$avg_glucose_level ~ Stroke_Data$gender, var.equal = FALSE)


	Welch Two Sample t-test

data:  Stroke_Data$avg_glucose_level by Stroke_Data$gender
t = -6.4663, df = 34587, p-value = 1.018e-10
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -3.590087 -1.919921
sample estimates:
mean in group Female   mean in group Male 
            102.5178             105.2728 


In [8]:
## example 2 with BMI
t.test(Stroke_Data$bmi, mu=mu_0, alternative="greater")


	One Sample t-test

data:  Stroke_Data$bmi
t = -36.759, df = 41930, p-value = 1
alternative hypothesis: true mean is greater than 30
95 percent confidence interval:
 28.54274      Inf
sample estimates:
mean of x 
 28.60516 


### <a id='pval'>P-value</a> 

The **p value** is the smallest value of $\alpha$ where we still reject a null hypothesis. The smaller the p-value, the strong the evidence that you should reject the null hypothesis.

> P-value is the probability under the null hypothesis of obtaining evidence as extreme or more extreme than would be observed by chance alone.

Consider an example:

* $H_0$ : ratio of strokes among men and women is the same

* $H_A$ : ratio of strokes among men and women is different

* $\alpha$ = 0.05

We will use `prop.test` function for this:

In [9]:
stroke <- Stroke_Data %>%
    select(gender, stroke) %>%
    mutate(stroke = as.integer(levels(Stroke_Data$stroke))[Stroke_Data$stroke]) %>%
    group_by(gender) %>%
    summarise(n = n(), # total number of observations
              x = sum(stroke)) # total number of positive stroke outcomes
stroke

gender,n,x
Female,24945,357
Male,16986,286


In [10]:
result <- prop.test(x = c(stroke$x[1], stroke$x[2]), 
                    n = c(stroke$n[1], stroke$n[2]), 
                    alternative = "two.sided")
result


	2-sample test for equality of proportions with continuity correction

data:  c(stroke$x[1], stroke$x[2]) out of c(stroke$n[1], stroke$n[2])
X-squared = 4.1042, df = 1, p-value = 0.04278
alternative hypothesis: two.sided
95 percent confidence interval:
 -5.007699e-03 -4.412189e-05
sample estimates:
    prop 1     prop 2 
0.01431149 0.01683740 


In [11]:
print(paste0("Calculated p-value is: ", round(result$p.value, 3)))
print(paste0("Significance level alpha is: 0.05"))

[1] "Calculated p-value is: 0.043"
[1] "Significance level alpha is: 0.05"


**We see that $p.value<\alpha$ so we reject the null hypothesis - there are not enough evidence to believe that ratio of strokes among men and women is the same.**

## <a id='power'>Statistical Power</a> 

Power is the probability of rejecting the null hypothesis whe it is false. **The more power, the better!**

$Power=1-\beta$

In a power calculation, one fixes the sample size and other aspects of the study and calculates an estimate of the power. In a sample size calculation, one fixes the power then determines the minimum sample size to acheive it.

### <a id='pwr'>Meet the pwr package</a> 

#### Find the Power value

Recall the last example with the stroke outcomes ratio among men and women.

Let's find the power of a test where we want to check whether the proportion between the groups is significant with significance level of 0.05. Let's say that proportion for the first group is 0.1 and 0.2 for the second group. Each group consist of 100 observations. 

We will use `pwr.2p2n.test` from `pwr` package. The amazing part of this package is that you have to fill in parameters you have (size of proportions, ratio) and it will calculate the rest (power).

In [12]:
stroke

gender,n,x
Female,24945,357
Male,16986,286


In [13]:
n1 <- stroke$n[1]
p1 <- stroke$x[1]/n1
n2 <- stroke$n[2]
p2 <- stroke$x[2]/n2
h <- p2 - p1 ## difference between ratios

In [14]:
pwr.2p2n.test(h = h, 
              n1 = n1, 
              n2 = n2, 
              sig.level = 0.05)


     difference of proportion power calculation for binomial distribution (arcsine transformation) 

              h = 0.00252591
             n1 = 24945
             n2 = 16986
      sig.level = 0.05
          power = 0.05741801
    alternative = two.sided

NOTE: different sample sizes


Power is just 5.7%. In other words, the probability of correctly rejecting null hypothesis (no difference between the groups) is 5.7%.

#### Find the best Sample Size

Now image you want to run a test to find "small" effect between two groups proportion (difference ~ 0.2). You want to know want sample size should you choose to get the oprimal value of power.

In [15]:
n <- seq(10,100,10)
p.out <- pwr.p.test(h = 0.2,
                    n = n,
                    sig.level = 0.05)
data.frame(n, power = sprintf("%.2f%%", p.out$power * 100))

n,power
10,9.69%
20,14.55%
30,19.48%
40,24.41%
50,29.30%
60,34.08%
70,38.73%
80,43.22%
90,47.51%
100,51.60%


In order to achive 51% probability of correctly rejecting null hypothesis you have to gather information from 2 groups with 100 people each.

**You can find more examples of `pwr` package usage on [Cran-R](https://cran.r-project.org/web/packages/pwr/vignettes/pwr-vignette.html).**

## <a id='next'>What to Expect Next</a>

Now I have covered all subjects I wanted. I will go through all them again later to check if there are any mistakes or to add some more examples. 

If you want to dig deeper into biostatistics I higly encourage you take two-part course at Coursera: [Mathematical Biostatistics Boot Camp 1](https://www.coursera.org/learn/biostatistics/) & [Mathematical Biostatistics Boot Camp 2](https://www.coursera.org/learn/biostatistics-2/) and go through [Methods in Biostatistics with R](https://leanpub.com/biostatmethods) book.

**So, what I expect from you now is to ask me questions about problems you want to solve with statistical analysis that were covered in this 3-part tutorial. Feel free to contact me here on Kaggle or Linkedin.**